In [1]:
import pandas as pd
from datetime import timedelta,datetime

In [2]:
df = pd.read_csv("order_brush_order.csv")
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [3]:
df['event_time'] = pd.to_datetime(df['event_time'],format='%Y-%m-%d %H:%M:%S')

In [4]:
shopids = df.shopid.unique()

In [5]:
len(shopids)

18770

In [6]:
result = pd.DataFrame(columns=['shopid','userid'])

In [7]:
from tqdm import notebook.tqdm

In [12]:
start

Timestamp('2019-12-28 03:16:09')

In [13]:
end

Timestamp('2019-12-28 04:16:09')

In [8]:

for shopid in notebook.tqdm(shopids):
    shopid0 = df[df.shopid==shopid].sort_values(by='event_time')
    shopid0.reset_index(inplace=True)
    i = 0
    brushing_periods = 0
    brushing_users = {}
    while i<shopid0.shape[0]:
        f = 0
        start = shopid0.event_time[i]
        end = start + timedelta(hours=1)

        period0 = shopid0[(start<=shopid0.event_time)&(shopid0.event_time<=end)]    
        brushing_period = len(period0)
        unique_user = period0.userid.nunique()
        if brushing_period/unique_user>=3:
            f = 1
            brushing_periods+=brushing_period
            d = period0.userid.value_counts().to_dict()
            for k in d:
                if k in brushing_users:
                    brushing_users[k]+=d[k]
                else:
                    brushing_users[k] = d[k]
        if f:
            i = period0.index[-1] +1
        else:
            i +=1
    userid = []
    if brushing_periods>0:
        k = sorted(brushing_users,key=lambda x:brushing_users[x],reverse=True)
        userid = [k[0]]
        for i in range(1,len(k)):
            if brushing_users[k[i]] == brushing_users[k[0]]:
                userid.append(k[i])
    userid = "&".join(map(str,userid))
    if len(userid) == 0:
        userid=0
    result = result.append(pd.DataFrame(data={'shopid':[shopid],'userid':[userid]}),ignore_index=True)

<ipython-input-8-fe0552bfaea4>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for shopid in tqdm_notebook(shopids):


In [9]:
result.to_csv("output.csv",index=False)

In [11]:
result[result.userid.astype(str) != "0"].count()

shopid    276
userid    276
dtype: int64